In [1]:
import tensorflow as tf
import numpy as np
import emnist
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from IPython.display import display, Image
from PIL import Image, ImageFilter

import numpy


subset = [0,1,2,3,4,5,6,7,8,9]  
mnist = emnist.read_data_sets('/Users/nghia/tmp/nghia/mnist', prefix="", validation_size=10, subset = subset) 
#mnist = emnist.read_data_sets('data/emnist', subset = subset)

#Debugging
print("shape:" + str(mnist.train.next_batch(50)[0].shape))

/Users/nghia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /Users/nghia/tmp/nghia/mnist/train-images-idx3-ubyte.gz
Numer of images:373
Number of rows and cols: 28, 28
Extracting /Users/nghia/tmp/nghia/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/nghia/tmp/nghia/mnist/test-images-idx3-ubyte.gz
Numer of images:373
Number of rows and cols: 28, 28
Extracting /Users/nghia/tmp/nghia/mnist/test-labels-idx1-ubyte.gz
Validation size=10
len(train_images)=373
Only retain label: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Labels:3
Labels:0
Labels:5
Labels:6
Labels:2
Labels:9
Labels:7
Labels:7
Labels:2
Labels:7
Labels:34
Labels:4
Labels:1
Labels:34
Labels:6
Labels:9
Labels:1
Labels:2
Labels:2
Labels:0
Labels:3
Labels:1
Labels:7
Labels:9
Labels:4
Labels:1
Labels:0
Labels:6
Labels:1
Labels:1
Labels:1
Labels:6
Labels:1
Labels:5
Labels:6
Labels:9
Labels:5
Labels:3
Labels:2
Labels:0
Labels:6
Labels:2
Labels:8
Labels:8
Labels:0
Labels:7
Labels:1
Labels:7
Labels:1
Labels:6
Labels:25
Labels:2
Labels:8
Labels:4
Labels:40
Labels:40
Labels:3
Labels:1
Labels:1
Lab

In [2]:
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)

def conv2d(x, W, name=None):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x, name):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

image_size = 28
num_label = len(subset)
num_steps = 200

graph = tf.Graph()
print("mnist.train.next_batch(50):" ,mnist.train.next_batch(50)[0].shape)
with graph.as_default():  
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
    y_ = tf.placeholder(tf.float32, shape=[None, num_label], name='y_')
    global_step = tf.Variable(0, name='global_step', trainable=False)
       
    W_conv1 = weight_variable([5, 5, 1, 32], 'W_conv1')
    b_conv1 = bias_variable([32], 'b_conv1')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')
                        

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, name='h_conv1')
    h_pool1 = max_pool_2x2(h_conv1, 'h_pool1')
    W_conv2 = weight_variable([5, 5, 32, 64], 'W_conv2')
    b_conv2 = bias_variable([64], 'b_conv2')
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2, 'h_pool2')
    W_fc1 = weight_variable([7 * 7 * 64, 1024], 'W_fc1')
    b_fc1 = bias_variable([1024], 'b_fc1')
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    W_fc2 = weight_variable([1024, num_label], 'W_fc2')
    b_fc2 = bias_variable([num_label], 'b_fc2')

    y_conv = tf.identity(tf.matmul(h_fc1_drop, W_fc2) + b_fc2,name='y_conv')
    
    
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), name='cross_entropy')
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy, name='train_step')
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    tf.add_to_collection('y_conv', y_conv)
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('cross_entropy', cross_entropy)
    tf.add_to_collection("train_step", train_step)

print('mnist.test_images.shape', mnist.test.images.shape)
print('mnist.test_labels.shape', mnist.test.labels.shape)

trained_filename = '/Users/nghiaround/rec_table/train/checkpoint/no'
#print('Label ', mnist.test.labels[0:10])

image_size = 28
one_image = mnist.train.images[20] 
show_image = one_image.reshape([image_size, image_size])
plt.imshow(show_image, aspect="auto") 

with tf.Session(graph=graph) as session:
    session.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    for i in range(num_steps):
        batch = mnist.train.next_batch(50)
        if i % 10 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    saver.save(session, trained_filename)
    print('After save data')

Start = 50 batch_size:50 num_examples:353
mnist.train.next_batch(50): (50, 784)
mnist.test_images.shape (363, 784)
mnist.test_labels.shape (363, 10)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Start = 100 batch_size:50 num_examples:353
step 0, training accuracy 0.06
Start = 150 batch_size:50 num_examples:353
Start = 200 batch_size:50 num_examples:353
Start = 250 batch_size:50 num_examples:353
Start = 300 batch_size:50 num_examples:353
Start = 350 batch_size:50 num_examples:353
Start = 47 batch_size:50 num_examples:353
Start = 97 batch_size:50 num_examples:353
Start = 147 batch_size:50 num_examples:353
Start = 197 batch_size:50 num_examples:353
Start = 247 batch_size:50 num_examples:353
step 10, training accuracy 0.22
Start = 297 batch_size:50 num_examples:353
Start = 347 batch_size:50 num_examples:353
Start = 44 batch_size:50 num_examples:353
Start = 94 batch_size:50 num_examples:353
Start = 144 batch_size:50 num_examples:353
Start = 194 batch_size:50 num_

Start = 78 batch_size:50 num_examples:353
Start = 128 batch_size:50 num_examples:353
step 170, training accuracy 0.98
Start = 178 batch_size:50 num_examples:353
Start = 228 batch_size:50 num_examples:353
Start = 278 batch_size:50 num_examples:353
Start = 328 batch_size:50 num_examples:353
Start = 25 batch_size:50 num_examples:353
Start = 75 batch_size:50 num_examples:353
Start = 125 batch_size:50 num_examples:353
Start = 175 batch_size:50 num_examples:353
Start = 225 batch_size:50 num_examples:353
Start = 275 batch_size:50 num_examples:353
step 180, training accuracy 0.98
Start = 325 batch_size:50 num_examples:353
Start = 22 batch_size:50 num_examples:353
Start = 72 batch_size:50 num_examples:353
Start = 122 batch_size:50 num_examples:353
Start = 172 batch_size:50 num_examples:353
Start = 222 batch_size:50 num_examples:353
Start = 272 batch_size:50 num_examples:353
Start = 322 batch_size:50 num_examples:353
Start = 19 batch_size:50 num_examples:353
Start = 69 batch_size:50 num_examples

KeyboardInterrupt: 